In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import distance
from sklearn.metrics import log_loss
from sklearn.neighbors import KNeighborsClassifier

In [2]:
train = pd.read_csv('train_prepared.csv', encoding='utf-8')
test = pd.read_csv('test_prepared.csv', encoding='utf-8')

In [3]:
X_train = train.drop(columns=['Target'])
y_train = train.Target

In [4]:
X_test = test.drop(columns=['Target'])
y_test = test.Target

In [5]:
X_train_normalized = (X_train - X_train.mean()) / X_train.std()
X_train_normalized.fillna(0.0, inplace=True)
X_train_normalized

,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Opportunity_ID,Delivery_Year,Convertibility,Total_Taxable_Amount_USD,Prod_Price_Mean,Prod_Price_Median,...,Bills_Same_Country,Or,Region_target,Territory_target,Bureaucratic_Code_target,Billing_Country_target,Account_Type_target,Opportunity_Type_target,Delivery_Terms_target,Total_Amount_Currency_target
0,0.558329,0.904684,1.002505,1.400814,-1.622161,-1.437477,0.665021,0.270641,0.622002,0.622281,...,-1.609387,-1.072095,-0.575829,0.115166,0.558543,-0.302604,-0.204755,0.012130,-0.582828,-0.435442
1,-1.790876,-1.105246,-0.997399,-0.713798,-1.621855,-1.437477,0.665016,-0.098801,-0.180329,-0.180138,...,-1.609387,-1.072095,-0.575829,0.115166,0.558543,-0.302604,-0.204755,0.012130,-0.582828,-0.435442
2,-1.790876,-1.105246,-0.997399,-0.713798,-1.621548,-1.437477,0.321141,-0.096969,-0.176349,-0.176158,...,0.621291,0.932658,-0.757380,0.755136,0.558543,-0.484694,-1.495015,0.012130,0.441919,-0.506469
3,0.558329,-1.105246,1.002505,-0.713798,-1.621241,1.042582,0.321141,0.361780,0.819932,0.820234,...,0.621291,0.932658,-0.757380,0.755136,-1.791629,-0.484694,-1.495015,-2.095686,-1.039125,-0.506469
4,0.558329,-1.105246,1.002505,-0.713798,-1.620934,1.042582,0.321141,0.732836,1.625769,1.626159,...,0.621291,0.932658,-0.757380,0.755136,-1.791629,-0.484694,-1.495015,-2.095686,-1.039125,-0.506469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9786,0.558329,0.904684,-0.997399,-0.713798,2.303833,-1.437477,0.664998,-0.073807,-0.126047,-0.125851,...,-1.609387,0.932658,-0.575829,-0.856972,0.558543,-0.043211,1.120885,0.012130,-0.582828,-0.435442
9787,0.558329,-1.105246,-0.997399,-0.713798,2.304140,-1.437477,0.321141,1.231427,1.260420,1.260770,...,-1.609387,-1.072095,-0.757380,-1.130678,-1.791629,-0.043211,-1.495015,-2.095686,0.441919,-0.506469
9788,0.558329,0.904684,1.002505,1.400814,2.304447,-1.437477,0.665021,-0.081018,-0.172393,-0.171673,...,0.621291,0.932658,-0.575829,1.897817,0.558543,1.951205,-1.495015,0.012130,-0.582828,-0.435442
9789,0.558329,0.904684,1.002505,1.400814,2.304754,-1.437477,0.321141,-0.102212,0.130889,0.131114,...,0.621291,-1.072095,-0.757380,-1.130678,0.558543,-0.484694,-1.495015,0.012130,0.441919,-0.506469


In [6]:
X_test_normalized = (X_test - X_test.mean()) / X_test.std()
X_test_normalized.fillna(0.0, inplace=True)
X_test_normalized

,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Opportunity_ID,Delivery_Year,Convertibility,Total_Taxable_Amount_USD,Prod_Price_Mean,Prod_Price_Median,...,Bills_Same_Country,Or,Region_target,Territory_target,Bureaucratic_Code_target,Billing_Country_target,Account_Type_target,Opportunity_Type_target,Delivery_Terms_target,Total_Amount_Currency_target
0,0.783204,1.011876,1.449699,1.928083,-1.757493,-0.18386,0.567386,-0.065856,-0.074831,-0.074569,...,-0.05656,0.792778,-0.387408,0.999681,0.425693,0.119289,1.122651,1.057467,-0.665258,-0.241485
1,0.783204,1.011876,1.449699,1.928083,-1.755461,-0.18386,0.567381,-0.041349,-0.072473,-0.074569,...,-0.05656,0.792778,-0.387408,0.999681,0.425693,0.119289,1.122651,1.057467,-0.665258,-0.241485
2,0.783204,1.011876,-0.689358,-0.518319,-1.753430,-0.18386,0.121598,-0.087754,-0.083252,-0.083320,...,-0.05656,0.792778,-0.612171,0.999681,0.425693,-0.391675,-0.382238,0.961684,0.394711,-0.352977
3,0.783204,1.011876,1.449699,-0.518319,-1.751398,-0.18386,0.121598,0.031468,-0.058848,-0.060283,...,-0.05656,0.792778,-0.612171,0.999681,-2.094038,-0.391675,1.122651,-0.202249,0.394711,-0.352977
4,-1.275992,-0.987632,-0.689358,-0.518319,-1.749366,-0.18386,0.121598,-0.088602,-0.084348,-0.084415,...,-0.05656,0.792778,-0.612171,0.999681,0.425693,-0.391675,1.122651,0.961684,0.394711,-0.352977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,0.783204,1.011876,-0.689358,-0.518319,1.645554,-0.18386,0.121598,-0.080721,-0.079465,-0.079533,...,-0.05656,0.792778,-0.387408,0.999681,0.425693,0.390857,1.122651,-0.202249,0.091383,-0.352977
1563,0.783204,1.011876,-0.689358,-0.518319,1.647586,-0.18386,0.121598,-0.086421,-0.081531,-0.081599,...,-0.05656,-1.260582,-0.612171,0.999681,0.425693,-0.391675,-0.382238,-0.202249,0.394711,-0.352977
1564,0.783204,1.011876,1.449699,1.928083,1.649617,-0.18386,0.121598,-0.083364,-0.077584,-0.077652,...,-0.05656,-1.260582,-0.612171,0.999681,0.425693,-0.391675,-0.382238,-0.202249,0.394711,-0.352977
1565,0.783204,1.011876,-0.689358,-0.518319,1.651649,-0.18386,0.121598,-0.080972,-0.074494,-0.074564,...,-0.05656,-1.260582,-0.612171,0.999681,0.425693,-0.391675,-0.382238,0.961684,0.394711,-0.352977


In [7]:
for k in range(1, 100):
    knn = KNeighborsClassifier(n_neighbors=k)
    # Fit the model on the training data.
    knn.fit(X_train_normalized, y_train)
    # Make point predictions on the test set using the fit model.
    predictions = knn.predict(X_test_normalized)
    pp = knn.predict_proba(X_test_normalized)
    print("k=" + str(k) + ": " + str(log_loss(y_test, pp)))
    print()

k=1: 13.819918825532225

k=2: 7.891189030734791

k=3: 5.01065875427317

k=4: 3.6412368861211855

k=5: 2.7836340150945635

k=6: 2.3654371620151213

k=7: 1.8261337327532638

k=8: 1.7158124964278123

k=9: 1.5557324983575358

k=10: 1.4485663408602585

k=11: 1.2843844157655102

k=12: 1.1995109845250436

k=13: 1.0745424581850878

k=14: 1.0328947362843282

k=15: 0.931936851704779

k=16: 0.8948508329652478

k=17: 0.8716922660771954

k=18: 0.8341088854617053

k=19: 0.8333375982822515

k=20: 0.8114666132725964

k=21: 0.789046363588334

k=22: 0.7905120226813153

k=23: 0.7894980806000106

k=24: 0.7688064911887015

k=25: 0.7466803389565806

k=26: 0.7249808981492524

k=27: 0.7227622092161888

k=28: 0.7211615273443711

k=29: 0.7219277475454767

k=30: 0.7207612690188061

k=31: 0.7209626465629433

k=32: 0.7210422478455336

k=33: 0.7202967889153468

k=34: 0.7010937213419521

k=35: 0.7024597688304394

k=36: 0.7017448826139969

k=37: 0.7009531137342732

k=38: 0.7019678409315289

k=39: 0.7008589361941184

